# Day 2 Dry Run on GPU
Generate tests for 5 bugs using A100 GPU

**IMPORTANT: Go to Runtime → Change runtime type → Select A100 GPU**

In [ ]:
# Verify GPU
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  No GPU detected! Change runtime type to A100")

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Extract code
import os
from pathlib import Path

work_dir = Path('/content/libro_replication')
work_dir.mkdir(exist_ok=True)
os.chdir(work_dir)

# Unzip code from Drive
!unzip -q /content/drive/MyDrive/libro_replication/libro_code.zip -d /content/libro_replication

print("✓ Code extracted")
!ls -la

In [ ]:
# Install dependencies
!pip install transformers torch accelerate huggingface_hub sentencepiece -q
!pip install tqdm pyyaml -q

print("✓ Dependencies installed")

In [ ]:
# HuggingFace login
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Imports
import sys
sys.path.append('/content/libro_replication')

import json
import logging
from pathlib import Path

from src.model_manager import ModelManager
from src.core.prompt_builder import PromptBuilder
from src.core.test_generator import TestGenerator
from src.core.batch_processor import BatchProcessor

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger(__name__)
print("✓ Imports successful")

In [ ]:
# Load bug reports
with open('data/bugs/bug_reports.json') as f:
    all_bug_reports = json.load(f)

# Select first 5 bugs for dry run
bug_reports = all_bug_reports[:5]

print(f"Loaded {len(bug_reports)} bugs for dry run:")
for bug in bug_reports:
    print(f"  - {bug['project']}-{bug['bug_id']}: {bug['title'][:60]}...")

In [ ]:
# Initialize Model (THIS WILL TAKE 5-10 MINUTES)
print("Loading model... (this will take 5-10 minutes)")

model_manager = ModelManager(
    model_key="starcoder2-15b",
    cache_dir="/content/drive/MyDrive/libro_replication/models/starcoder2-15b"
)

model_manager.load()

print("✓ Model loaded successfully!")
print(model_manager.get_info())

In [ ]:
# Initialize prompt builder
prompt_builder = PromptBuilder(
    num_examples=2,
    examples_file="data/examples/manual_examples.json"
)

print(f"✓ Prompt builder initialized with {len(prompt_builder.examples)} examples")

In [ ]:
# Initialize test generator
test_generator = TestGenerator(
    model_manager=model_manager,
    cache_dir="/content/drive/MyDrive/libro_replication/cache/generations"
)

print("✓ Test generator initialized")

In [ ]:
# Initialize batch processor
batch_processor = BatchProcessor(
    prompt_builder=prompt_builder,
    test_generator=test_generator,
    output_dir="/content/drive/MyDrive/libro_replication/results/dry_run_gpu"
)

print("✓ Batch processor initialized")

In [ ]:
# Run generation (10-15 minutes)
print("Starting generation...")
print("This will take approximately 10-15 minutes")
print()

results = batch_processor.process_bugs(
    bug_reports=bug_reports,
    n_samples=10
)

print("\n✓ Generation complete!")

In [ ]:
# Summary statistics
total_bugs = len(results)
total_tests = sum(len(tests) for tests in results.values())
bugs_with_tests = sum(1 for tests in results.values() if tests)

print("=" * 80)
print("DRY RUN RESULTS SUMMARY")
print("=" * 80)
print(f"\nTotal bugs processed: {total_bugs}")
print(f"Bugs with ≥1 test: {bugs_with_tests} ({bugs_with_tests/total_bugs*100:.1f}%)")
print(f"Total tests generated: {total_tests}")
print(f"Average tests per bug: {total_tests/total_bugs:.1f}")

In [ ]:
# Show sample tests
print("\n" + "=" * 80)
print("SAMPLE TESTS")
print("=" * 80)

for bug_id, tests in results.items():
    print(f"\n{bug_id}: {len(tests)} tests generated")
    
    if tests:
        print("  First test:")
        print("  " + "-" * 70)
        for line in tests[0]['test_code'].split('\n')[:15]:
            print(f"  {line}")
        if len(tests[0]['test_code'].split('\n')) > 15:
            print("  ...")
        print(f"\n  Generation time: {tests[0].get('generation_time', 0):.2f}s")

In [ ]:
# Quality analysis
import re

def check_quality(test_code):
    return {
        'has_signature': bool(re.search(r'public\s+void\s+test', test_code)),
        'has_assertion': bool(re.search(r'assert\w+\(', test_code, re.IGNORECASE)),
        'balanced_braces': test_code.count('{') == test_code.count('}'),
        'has_content': len(test_code.strip()) > 50
    }

print("\n" + "=" * 80)
print("QUALITY ANALYSIS")
print("=" * 80)

all_tests = [test for tests in results.values() for test in tests]
quality_scores = [sum(check_quality(t['test_code']).values()) for t in all_tests]

print(f"\nQuality distribution (out of 4 checks):")
print(f"  Perfect (4/4): {sum(1 for s in quality_scores if s == 4)} tests")
print(f"  Good (3/4): {sum(1 for s in quality_scores if s == 3)} tests")
print(f"  Acceptable (2/4): {sum(1 for s in quality_scores if s == 2)} tests")
print(f"  Poor (<2/4): {sum(1 for s in quality_scores if s < 2)} tests")

In [ ]:
# Verify files saved to Drive
output_dir = Path("/content/drive/MyDrive/libro_replication/results/dry_run_gpu")

print("Files saved to Google Drive:")
for file in output_dir.glob("*"):
    print(f"  ✓ {file.name}")

print("\nYou can find these files in:")
print("  Google Drive → libro_replication → results → dry_run_gpu/")

In [ ]:
# Create zip for easy download
!cd /content/drive/MyDrive/libro_replication/results && zip -r dry_run_results.zip dry_run_gpu/

print("✓ Results zipped")
print("Download from: Google Drive → libro_replication → results → dry_run_results.zip")

## Summary

✅ **Completed:**
- Generated tests for 5 bugs
- 10 samples per bug
- Used StarCoder2-15B on A100 GPU
- Results saved to Google Drive

**Files in Google Drive:**
- `generation_results.json` - Raw generation data
- `generation_stats.json` - Summary statistics
- `progress.json` - Progress tracking
- Individual bug caches in `cache/generations/`

**Next Steps:**
- Day 3: Implement test injection and execution
- Download results and continue locally or expand to 30 bugs